In [384]:
import pandas as pd

In [385]:
# import submission directory data
illumina_parent = pd.read_csv('PARENTS_1000G_DATA.csv',header=None)
illumina_children = pd.read_csv('CHILDREN_1000G_DATA.csv', header=None)
illumina_y2 = pd.read_csv('Y2_1000G_DATA.csv', header=None)
illumina_y4 = pd.read_csv('Y4_1000G_DATA.csv', header=None)

illumina_pedigree = pd.read_csv('/Users/apblair/Desktop/projects/HPRC_metadata/data/hprc-data-explorer-tables/20130606_g1k.ped',sep='\t')

illumina_pedigree.rename(columns={'Individual ID':'sample_id',
								  'Paternal ID': 'paternal_id',
								  'Maternal ID': 'maternal_id'
								  }, inplace=True)

illumina_table = pd.concat([illumina_parent, illumina_children, illumina_y2, illumina_y4])
illumina_table = illumina_table[illumina_table[0].str.contains('final.cram') & ~illumina_table[0].str.contains('.md5') & ~illumina_table[0].str.contains('.cram.crai')]

illumina_table['sample_id'] = [sample_id.split('/')[-1].split('.')[0] for sample_id in illumina_table[0].tolist()]

# merge submission directory with illumina pedigree
illumina_table = pd.merge(illumina_table, 
						  illumina_pedigree[['sample_id','Family ID','maternal_id','paternal_id','Relationship']],
						  on='sample_id', how='inner')

child_working = []
for sample_id in illumina_table['sample_id'].tolist():
	sample_file = sample_id + '.final.cram'
	child_working.append(f"s3://human-pangenomics/working/HPRC/{sample_id}/raw_data/Illumina/child/{sample_file}")
illumina_table['aws-working-child'] = child_working

# create pedigree table
illumina_table_pedigree = illumina_table[(illumina_table['paternal_id'] != '0') \
												  & (illumina_table['maternal_id'] != '0')]

# Add working path
maternal_path = []
for data in illumina_table_pedigree[['sample_id','maternal_id']].values:
	sample_id = data[0]
	maternal_id = data[1]
	maternal_file = data[1] + '.final.cram'
	maternal_path.append(f"s3://human-pangenomics/working/HPRC/{sample_id}/raw_data/Illumina/parents/{maternal_id}/{maternal_file}")

paternal_path = []
for data in illumina_table_pedigree[['sample_id','paternal_id']].values:
	sample_id = data[0]
	paternal_id = data[1]
	paternal_file = data[1] + '.final.cram'
	paternal_path.append(f"s3://human-pangenomics/working/HPRC/{sample_id}/raw_data/Illumina/parents/{paternal_id}/{paternal_file}")

illumina_table_pedigree['aws-working-paternal'] = paternal_path
illumina_table_pedigree['aws-working-maternal'] = maternal_path

illumina_table_pedigree.rename(columns={0:'aws-submission-child'},inplace=True)

/var/folders/y2/lhjpt8gd3ss84zc6mhs151lc0000gn/T/ipykernel_429/244472216.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  illumina_table_pedigree['aws-working-paternal'] = paternal_path
/var/folders/y2/lhjpt8gd3ss84zc6mhs151lc0000gn/T/ipykernel_429/244472216.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  illumina_table_pedigree['aws-working-maternal'] = maternal_path
/var/folders/y2/lhjpt8gd3ss84zc6mhs151lc0000gn/T/ipykernel_429/244472216.py:52: SettingWithCopyWarning: 
A value is trying to be set

In [313]:
# list comprehension
# illumina_table_pedigree[~illumina_table_pedigree['maternal_id'].isin(illumina_table[illumina_table['sample_id'].isin(illumina_table_pedigree['maternal_id'].tolist())].sample_id.tolist())].shape

# missing_maternal = []
# for data in illumina_table_pedigree['maternal_id'].tolist():
# 	if data not in illumina_table['sample_id'].tolist():
# 		missing_maternal.append(data)
# print(len(missing_maternal))

# missing_paternal = []
# for data in illumina_table_pedigree['paternal_id'].tolist():

# 	if data not in illumina_table['sample_id'].tolist():
# 		missing_paternal.append(data)
# print(len(missing_paternal))
# missing 46 parents (both maternal and paternal samples)

# illumina_table_pedigree = illumina_table_pedigree[~illumina_pedigree['maternal_id'].isin(missing_maternal)] 
# illumina_table_pedigree = illumina_table_pedigree[~illumina_pedigree['paternal_id'].isin(missing_paternal)] 

In [386]:
maternal_submission_dict = {}
for data in illumina_table_pedigree['maternal_id'].tolist():
	if illumina_table[illumina_table['sample_id'].isin([data])][0].tolist() != []:
		maternal_submission_dict[data] = illumina_table[illumina_table['sample_id'].isin([data])][0].tolist()[0]
pedigree_maternal_df = pd.DataFrame.from_dict(maternal_submission_dict, orient='index')

parental_submission_dict = {}
for data in illumina_table_pedigree['paternal_id'].tolist():
	if illumina_table[illumina_table['sample_id'].isin([data])][0].tolist() != []:
		parental_submission_dict[data] = illumina_table[illumina_table['sample_id'].isin([data])][0].tolist()[0]
pedigree_paternal_df = pd.DataFrame.from_dict(parental_submission_dict, orient='index')

In [387]:
# subset down to the paternal and maternal 
illumina_table_pedigree = illumina_table_pedigree[illumina_table_pedigree['maternal_id'].isin(pedigree_maternal_df.index.tolist())]
# subset down to the paternal and maternal 
illumina_table_pedigree = illumina_table_pedigree[illumina_table_pedigree['paternal_id'].isin(pedigree_paternal_df.index.tolist())]

In [388]:
illumina_table_pedigree.head()

,aws-submission-child,sample_id,Family ID,maternal_id,paternal_id,Relationship,aws-working-child,aws-working-paternal,aws-working-maternal
36,s3://human-pangenomics/submissions/325b4b1c-9f...,HG00423,SH007,HG00422,HG00421,child,s3://human-pangenomics/working/HPRC/HG00423/ra...,s3://human-pangenomics/working/HPRC/HG00423/ra...,s3://human-pangenomics/working/HPRC/HG00423/ra...
41,s3://human-pangenomics/submissions/325b4b1c-9f...,HG00733,PR05,HG00732,HG00731,child,s3://human-pangenomics/working/HPRC/HG00733/ra...,s3://human-pangenomics/working/HPRC/HG00733/ra...,s3://human-pangenomics/working/HPRC/HG00733/ra...
46,s3://human-pangenomics/submissions/325b4b1c-9f...,HG01109,PR33,HG01108,HG01107,child,s3://human-pangenomics/working/HPRC/HG01109/ra...,s3://human-pangenomics/working/HPRC/HG01109/ra...,s3://human-pangenomics/working/HPRC/HG01109/ra...
49,s3://human-pangenomics/submissions/325b4b1c-9f...,HG01243,PR22,HG01242,HG01241,child,s3://human-pangenomics/working/HPRC/HG01243/ra...,s3://human-pangenomics/working/HPRC/HG01243/ra...,s3://human-pangenomics/working/HPRC/HG01243/ra...
61,s3://human-pangenomics/submissions/325b4b1c-9f...,HG02027,VN050,HG02028,HG02029,child,s3://human-pangenomics/working/HPRC/HG02027/ra...,s3://human-pangenomics/working/HPRC/HG02027/ra...,s3://human-pangenomics/working/HPRC/HG02027/ra...


In [389]:
illumina_table_pedigree['aws-submission-paternal'] = [parental_submission_dict[sample_id] for sample_id \
													  in illumina_table_pedigree['paternal_id'].tolist()]


illumina_table_pedigree['aws-submission-maternal'] = [maternal_submission_dict[sample_id] for sample_id \
													  in illumina_table_pedigree['maternal_id'].tolist()]


In [390]:
illumina_table_pedigree.to_csv('Illumina_pedigree.csv',index=False)

In [391]:
illumina_table_pedigree.shape

(123, 11)

In [372]:
# illumina_table_pedigree[illumina_table_pedigree['sample_id'] == 'HG00438']

In [392]:
illumina_table = illumina_table[~illumina_table['sample_id'].isin(illumina_table_pedigree['sample_id'].tolist())]

In [393]:
illumina_table.rename(columns={0:'aws-submission-child'},inplace=True)

In [394]:
illumina_table.sample_id.nunique()

383

In [376]:
illumina_table.to_csv('Illumina_Y1-Y4.transfer.csv',index=False)

,aws-submission-child,sample_id,Family ID,maternal_id,paternal_id,Relationship,aws-working-child
0,s3://human-pangenomics/submissions/325b4b1c-9f...,HG00099,HG00099,0,0,unrel,s3://human-pangenomics/working/HPRC/HG00099/ra...
1,s3://human-pangenomics/submissions/325b4b1c-9f...,HG00140,HG00140,0,0,unrel,s3://human-pangenomics/working/HPRC/HG00140/ra...
2,s3://human-pangenomics/submissions/325b4b1c-9f...,HG00280,HG00280,0,0,unrel,s3://human-pangenomics/working/HPRC/HG00280/ra...
3,s3://human-pangenomics/submissions/325b4b1c-9f...,HG00323,HG00323,0,0,unrel,s3://human-pangenomics/working/HPRC/HG00323/ra...
5,s3://human-pangenomics/submissions/325b4b1c-9f...,HG00438,SH012,HG00437,HG00436,child,s3://human-pangenomics/working/HPRC/HG00438/ra...
...,...,...,...,...,...,...,...
469,s3://human-pangenomics/submissions/59C50DDF-5F...,NA21101,NA21101,0,0,unrel,s3://human-pangenomics/working/HPRC/NA21101/ra...
470,s3://human-pangenomics/submissions/59C50DDF-5F...,NA21102,NA21102,0,0,unrel,s3://human-pangenomics/working/HPRC/NA21102/ra...
471,s3://human-pangenomics/submissions/59C50DDF-5F...,NA21106,NA21106,0,0,unrel,s3://human-pangenomics/working/HPRC/NA21106/ra...
472,s3://human-pangenomics/submissions/59C50DDF-5F...,NA21110,NA21110,0,0,unrel,s3://human-pangenomics/working/HPRC/NA21110/ra...


In [330]:
# illumina_table_pedigree[['aws-working-child', 'aws-working-paternal','aws-working-maternal']]